In [1]:
import sqlite3

from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from scipy.stats import fisk

In [9]:
class DataLoader:
    """
    Class responsible for loading data from an SQLite database.
    """

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.dataframes = {}

    def load_data(self) -> dict:
        """
        Load data from the SQLite database and store it in a dictionary.

        Returns:
            dict: A dictionary where keys are table names and values are DataFrames.
        """
        connection = sqlite3.connect(self.db_path)
        tables = [
            "drivers",
            "fcyphases",
            "laps",
            "qualifyings",
            "races",
            "retirements",
            "starterfields",
        ]
        self.dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", connection)
            for table in tables
        }
        connection.close()
        return self.dataframes



In [10]:
db_path = "F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()

In [8]:
laps_df = dataframes["laps"]
laps_race = laps_df[laps_df["race_id"] == 2]

# Trouver le dernier tour de chaque pilote
last_laps = laps_race.groupby("driver_id")["lapno"].max().reset_index()

# Récupérer le temps de course associé au dernier tour de chaque pilote
final_times = pd.merge(laps_race, last_laps, on=["driver_id", "lapno"], how="inner")[["driver_id", "racetime"]]
final_times = final_times.rename(columns={"racetime": "cumulative_time_actual"})


In [9]:
final_times

,driver_id,cumulative_time_actual
0,16,0.000
1,21,808.607
2,18,1029.632
3,6,2089.905
4,13,3565.971
5,20,3897.937
6,2,5522.546
7,19,6037.511
8,17,6037.641
9,4,6028.365


In [26]:
import pandas as pd
import numpy as np

def generate_pit_stop_strategy(season: int, location: str, dataframes: dict):
    """
    Generates pit stop strategies for each driver in a given season and location.
    
    Args:
        season (int): The racing season.
        location (str): Grand Prix location.
        dataframes (dict): Dictionary containing race-related data.
    
    Returns:
        dict: Pit stop strategies for each driver.
    """
    # Load required data from the dataframes
    df_pit_stops = dataframes.get("laps", pd.DataFrame())
    df_races = dataframes.get("races", pd.DataFrame())
    df_drivers = dataframes.get("drivers", pd.DataFrame())
    
    # Find the race ID for the given season and location
    race_row = df_races[(df_races["season"] == season) & (df_races["location"] == location)]
    if race_row.empty:
        raise ValueError(f"No race found for location '{location}' in season {season}.")
    
    race_id = race_row.iloc[0]["id"]
    
    # Filter pit stop data for the selected race
    pit_stop_data = df_pit_stops[df_pit_stops["race_id"] == race_id]
    
    # Extract unique drivers from the filtered data
    drivers = pit_stop_data["driver_id"].unique()
    
    pit_stop_strategies = {}
    
    for driver_id in drivers:
        driver_pit_stops = pit_stop_data[pit_stop_data["driver_id"] == driver_id]
        driver_name = df_drivers[df_drivers["id"] == driver_id]["name"].values[0]
        
        # Find the starting tire compound and tire age (lap 0)
        starting_tire = driver_pit_stops[driver_pit_stops["lapno"] == 0]
        if not starting_tire.empty:
            starting_compound = starting_tire.iloc[0]["compound"]
            starting_tire_age = starting_tire.iloc[0]["tireage"]
        else:
            starting_compound = "Unknown"
            starting_tire_age = "Unknown"
        
        strategy = {"starting_compound": starting_compound, "starting_tire_age": starting_tire_age}
        
        # Filter only actual pit stops (where pitstopduration is not NaN or zero)
        valid_pit_stops = driver_pit_stops[driver_pit_stops["pitstopduration"].notna() & (driver_pit_stops["pitstopduration"] > 0)]
        
        for idx, (lap, duration, compound, tireage) in enumerate(
            zip(valid_pit_stops["lapno"], valid_pit_stops["pitstopduration"], valid_pit_stops["compound"], valid_pit_stops["tireage"])
        ):
            pitstop_interval = [lap,lap]
            strategy[idx+1] = {
                "compound": compound,
                "pitstop_interval": pitstop_interval,
                "pit_stop_lap": lap,
                "tire_age": tireage
            }
        
        pit_stop_strategies[driver_name] = strategy
    
    return pit_stop_strategies

generate_pit_stop_strategy(season=2016, location="Austin",dataframes=dataframes)


{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tire_age': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tire_age': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tire_age': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tire_age': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tire_age': 11},
  2: {'comp

In [ ]:
{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tireage': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tireage': 11},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 17}},
 'Kimi Raikkonen': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [24, 24],
   'pit_stop_lap': 24,
   'tireage': 16},
  3: {'compound': 'A4',
   'pitstop_interval': [38, 38],
   'pit_stop_lap': 38,
   'tireage': 14}},
 'Sebastian Vettel': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [14, 14],
   'pit_stop_lap': 14,
   'tireage': 16},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 15},
  3: {'compound': 'A2',
   'pitstop_interval': [53, 53],
   'pit_stop_lap': 53,
   'tireage': 24}},
 'Nico Hulkenberg': {'starting_compound': 'A4', 'starting_tire_age': 2},
 'Valtteri Bottas': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [1, 1],
   'pit_stop_lap': 1,
   'tireage': 3},
  2: {'compound': 'A3',
   'pitstop_interval': [20, 20],
   'pit_stop_lap': 20,
   'tireage': 19}},
 'Felipe Massa': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 18},
  3: {'compound': 'A2',
   'pitstop_interval': [54, 54],
   'pit_stop_lap': 54,
   'tireage': 25}},
 'Carlos Sainz Jnr': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Sergio Perez': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Fernando Alonso': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 11},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Daniil Kvyat': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [21, 21],
   'pit_stop_lap': 21,
   'tireage': 21}},
 'Esteban Gutierrez': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13}},
 'Jolyon Palmer': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [15, 15],
   'pit_stop_lap': 15,
   'tireage': 15},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 11}},
 'Marcus Ericsson': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17}},
 'Romain Grosjean': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Kevin Magnussen': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 14},
  3: {'compound': 'A2',
   'pitstop_interval': [43, 43],
   'pit_stop_lap': 43,
   'tireage': 16}},
 'Jenson Button': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [28, 28],
   'pit_stop_lap': 28,
   'tireage': 18}},
 'Pascal Wehrlein': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 17}},
 'Felipe Nasr': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 29}},
 'Esteban Ocon': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 9},
  3: {'compound': 'A3',
   'pitstop_interval': [44, 44],
   'pit_stop_lap': 44,
   'tireage': 18}}}

In [8]:
from monte_carlo_simulator import MonteCarloSimulator

db_path = "F1_timingdata_2014_2019.sqlite"
season = 2016
gp_location = "Austin"
num_simulations = 2000

driver_strategies = {
    'Lewis Hamilton': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [31, 31],
                            'pit_stop_lap': 31,
                            'tire_age': 20}},
    'Nico Rosberg': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 12},
                        2: {'compound': 'A2',
                            'pitstop_interval': [31, 31],
                            'pit_stop_lap': 31,
                            'tire_age': 21}},
    'Daniel Ricciardo': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [8, 8],
                                'pit_stop_lap': 8,
                                'tire_age': 10},
                            2: {'compound': 'A3',
                                'pitstop_interval': [25, 25],
                                'pit_stop_lap': 25,
                                'tire_age': 17}},
    'Max Verstappen': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [9, 9],
                            'pit_stop_lap': 9,
                            'tire_age': 11},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 17}},
    'Kimi Raikkonen': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [8, 8],
                            'pit_stop_lap': 8,
                            'tire_age': 10},
                        2: {'compound': 'A3',
                            'pitstop_interval': [24, 24],
                            'pit_stop_lap': 24,
                            'tire_age': 16},
                        3: {'compound': 'A4',
                            'pitstop_interval': [38, 38],
                            'pit_stop_lap': 38,
                            'tire_age': 14}},
    'Sebastian Vettel': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [14, 14],
                                'pit_stop_lap': 14,
                                'tire_age': 16},
                            2: {'compound': 'A3',
                                'pitstop_interval': [29, 29],
                                'pit_stop_lap': 29,
                                'tire_age': 15},
                            3: {'compound': 'A2',
                                'pitstop_interval': [53, 53],
                                'pit_stop_lap': 53,
                                'tire_age': 24}},
    'Nico Hulkenberg': {'starting_compound': 'A4', 'starting_tire_age': 2},
    'Valtteri Bottas': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [1, 1],
                            'pit_stop_lap': 1,
                            'tire_age': 3},
                        2: {'compound': 'A3',
                            'pitstop_interval': [20, 20],
                            'pit_stop_lap': 20,
                            'tire_age': 19}},
    'Felipe Massa': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [29, 29],
                            'pit_stop_lap': 29,
                            'tire_age': 18},
                        3: {'compound': 'A2',
                            'pitstop_interval': [54, 54],
                            'pit_stop_lap': 54,
                            'tire_age': 25}},
    'Carlos Sainz Jnr': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [11, 11],
                                'pit_stop_lap': 11,
                                'tire_age': 13},
                            2: {'compound': 'A3',
                                'pitstop_interval': [30, 30],
                                'pit_stop_lap': 30,
                                'tire_age': 19}},
    'Sergio Perez': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A2',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 17}},
    'Fernando Alonso': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 11},
                        2: {'compound': 'A2',
                            'pitstop_interval': [30, 30],
                            'pit_stop_lap': 30,
                            'tire_age': 19}},
    'Daniil Kvyat': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [21, 21],
                            'pit_stop_lap': 21,
                            'tire_age': 21}},
    'Esteban Gutierrez': {'starting_compound': 'A3',
                            'starting_tire_age': 0,
                            1: {'compound': 'A3',
                                'pitstop_interval': [13, 13],
                                'pit_stop_lap': 13,
                                'tire_age': 13}},
    'Jolyon Palmer': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [15, 15],
                            'pit_stop_lap': 15,
                            'tire_age': 15},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 11}},
    'Marcus Ericsson': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [17, 17],
                            'pit_stop_lap': 17,
                            'tire_age': 17}},
    'Romain Grosjean': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A3',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 17}},
    'Kevin Magnussen': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [13, 13],
                            'pit_stop_lap': 13,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 14},
                        3: {'compound': 'A2',
                            'pitstop_interval': [43, 43],
                            'pit_stop_lap': 43,
                            'tire_age': 16}},
    'Jenson Button': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A2',
                            'pitstop_interval': [28, 28],
                            'pit_stop_lap': 28,
                            'tire_age': 18}},
    'Pascal Wehrlein': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [13, 13],
                            'pit_stop_lap': 13,
                            'tire_age': 13},
                        2: {'compound': 'A2',
                            'pitstop_interval': [30, 30],
                            'pit_stop_lap': 30,
                            'tire_age': 17}},
    'Felipe Nasr': {'starting_compound': 'A2',
                    'starting_tire_age': 0,
                    1: {'compound': 'A2',
                        'pitstop_interval': [29, 29],
                        'pit_stop_lap': 29,
                        'tire_age': 29}},
    'Esteban Ocon': {'starting_compound': 'A2',
                        'starting_tire_age': 0,
                        1: {'compound': 'A2',
                            'pitstop_interval': [17, 17],
                            'pit_stop_lap': 17,
                            'tire_age': 17},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 9},
                        3: {'compound': 'A3',
                            'pitstop_interval': [44, 44],
                            'pit_stop_lap': 44,
                            'tire_age': 18}}
    }



In [ ]:
# RUN Simulation 
simulator = MonteCarloSimulator(season, gp_location, db_path, driver_strategies, num_simulations)
simulator.run_simulation()
simulator.summarize()  # Compute comparison dataframe

simulator.comparison_df


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 1/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 2/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


...

c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 4/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 5/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


In [5]:
comparison_dfTest=simulator.comparison_df
comparison_dfTest.sort_values(by=["final_position_actual"])

,driver_id,final_position_sim,cumulative_time_sim,final_position_actual,cumulative_time_actual
0,1,2.5,5730.374037,1,5892.618
2,3,8.0,5804.125309,2,5897.138
1,2,6.5,5785.029074,3,5912.310
10,12,9.0,5832.871012,4,5935.752
4,5,15.0,5893.887158,5,5986.571
16,25,15.5,5900.093889,6,5988.742
7,9,5.5,5764.240632,7,5899.087
12,16,17.5,5956.275724,8,5913.660
8,10,18.0,6033.703596,9,5915.617
15,22,14.5,5889.280708,10,5923.518
